# LSTM Autoencoder Implementation
This is the notebook for the autoencoder we will use for automatic feature extraction. At the moment it is more of a test, than actual implementation. 

May 29th comment: This notebook ended up not being used. See Transformer.ipynb for extraction of automatic encodings. 

In [1]:
import numpy as np
import pandas as pd
import re
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec, Phrases

## Functions
Functions for processing some data beforehand...

In [4]:
def remove_dots(word):
    return re.sub(r',|\.|:|!|\?|;', '', word)

def split_words(text):
    '''Returns an array with text split into words'''
    text = text.lower()
    string = text.split()
    no_dot = [remove_dots(x) for x in string]
    return no_dot

## Word2Vec vectorization

In [9]:
#Edit these values to your liking.
vector_size = 200
min_count = 20
workers = 4
window = 10
sample = 1e-3

In [10]:
#Load data - headers for now.
with open('headers.txt', 'r', encoding='utf-8') as f:
    headers = f.readlines()
    headers = [x.replace('\n','') for x in headers]

headers = [split_words(x) for x in tqdm(headers)]
X_train, X_test = train_test_split(headers) #X_test not vectorized for now

  0%|          | 0/299025 [00:00<?, ?it/s]

In [11]:
%%time
bigram = Phrases(X_train)
trigram = Phrases(bigram[X_train])

Wall time: 7.36 s


In [12]:
%%time
#trigram[bigram[X_train]] Smid ind, når der skiftes til Body
model_bigram = Word2Vec(X_train, workers=workers, vector_size=vector_size,
                       min_count=min_count, window=window, sample=sample)
model_bigram.init_sims()

Wall time: 5.67 s


c:\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  after removing the cwd from sys.path.


In [13]:
vectors = model_bigram.wv

In [68]:
vector_dict = {}
vector_matrix
for idx, key in enumerate(vectors.key_to_index):
    vector_dict[idx] = vectors[key]
vector_matrix = np.array(list(vector_dict.values()))
vocab_size = len(vectors.key_to_index)

# Autoencoder

In [69]:
from keras.layers import Input, Embedding, Bidirectional, LSTM, GlobalMaxPooling1D, Dense, RepeatVector, TimeDistributed
from keras import regularizers
from keras.models import Model
import keras

In [87]:
inp = Input(shape=(vector_size,1)) #Vector_size, step_size instead?
 #Embedding?
encoder = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(inp)
encoder = Bidirectional(LSTM( 64, activation='relu', return_sequences=False))(encoder) #return state?

decoder = RepeatVector(vector_size)(encoder)
decoder = Bidirectional(LSTM( 64, activation='relu', return_sequences=True))(decoder) #initial state?
decoder = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(decoder)

decoder_output = TimeDistributed(Dense(vector_size, activation=None))(decoder)

autoencoder = Model(inputs=inp, outputs=decoder_output)
autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss='mse', metrics=['mse'])
autoencoder.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 200, 1)]          0         
_________________________________________________________________
bidirectional_41 (Bidirectio (None, 200, 256)          133120    
_________________________________________________________________
bidirectional_42 (Bidirectio (None, 128)               164352    
_________________________________________________________________
repeat_vector_9 (RepeatVecto (None, 200, 128)          0         
_________________________________________________________________
bidirectional_43 (Bidirectio (None, 200, 128)          98816     
_________________________________________________________________
bidirectional_44 (Bidirectio (None, 200, 256)          263168    
_________________________________________________________________
time_distributed_9 (TimeDist (None, 200, 200)          5140

In [88]:
input_matrix = np.expand_dims(vector_matrix, axis=2) #??
autoencoder.fit(input_matrix, input_matrix, epochs=5, batch_size=64)

Epoch 1/5
138/138 [==============================] - 203s 1s/step - loss: 0.0497 - mse: 0.0497
Epoch 2/5
138/138 [==============================] - 210s 2s/step - loss: 0.0497 - mse: 0.0497
Epoch 3/5
138/138 [==============================] - 222s 2s/step - loss: 0.0497 - mse: 0.0497
Epoch 4/5
138/138 [==============================] - 221s 2s/step - loss: 0.0497 - mse: 0.0497
Epoch 5/5
138/138 [==============================] - 212s 2s/step - loss: 0.0497 - mse: 0.0497


In [85]:
np.expand_dims(vector_matrix, axis=2).shape

(8823, 200, 1)

In [105]:
pred = autoencoder.predict(input_matrix[3])

In [106]:
vectors.most_similar(pred[0])

[('vejret', 0.168649360537529),
 ('play', 0.15579655766487122),
 ('ol', 0.15548251569271088),
 ('lift', 0.15332524478435516),
 ('royal', 0.14815519750118256),
 ('resultat', 0.14583541452884674),
 ('hav', 0.14575155079364777),
 ('nr', 0.14527243375778198),
 ('odense', 0.1450156718492508),
 ('løft', 0.13693858683109283)]

In [108]:
input_matrix[1]

array([[-0.03538927],
       [-0.83774674],
       [-0.5237304 ],
       [-0.36085996],
       [ 0.60985583],
       [ 0.67223126],
       [-0.21220826],
       [-0.1438552 ],
       [-0.44730672],
       [-0.30529448],
       [-0.0538767 ],
       [ 0.20183095],
       [-0.05600531],
       [ 0.4652583 ],
       [ 0.5390542 ],
       [-0.4623847 ],
       [ 0.90138906],
       [ 1.4761131 ],
       [-0.0810765 ],
       [-1.3600254 ],
       [ 0.00536684],
       [-0.8819717 ],
       [-1.0364987 ],
       [-0.367825  ],
       [ 0.7923582 ],
       [-0.13464624],
       [ 0.51270646],
       [ 0.63217   ],
       [-0.1583858 ],
       [ 0.59974915],
       [ 0.8403504 ],
       [-0.42354998],
       [-1.132408  ],
       [-0.11255053],
       [-0.46761727],
       [-0.10844474],
       [-0.28162044],
       [-0.1653496 ],
       [-0.02107022],
       [-0.06210085],
       [-0.6008779 ],
       [-0.4125722 ],
       [-0.54658145],
       [ 0.52796745],
       [ 0.69067633],
       [ 1

In [6]:
import os

In [7]:
path = 'data/additional/scraped'
files = os.listdir(path)
dataframes = []
for file in tqdm(files):
    dataframes.append(pd.read_json(path+'/'+file))

data = pd.concat(dataframes)
del data['level_0']
del data['index']

  0%|          | 0/31 [00:00<?, ?it/s]

In [21]:
#Write headers to file.
text = list(data.Body.astype(str))
texts = text
final_text = ''
for text in tqdm(texts):
    text = text.strip()
    text = text.replace('\n','')
    text = text.replace('\r','')
    text = text.replace('\t','') 
    if len(text) < 10:
        continue
    final_text += ''.join(text)
    final_text += '\n'
final_text = final_text[:-1]

  0%|          | 0/300822 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
len(final_text.split('\n'))